In [7]:
# import unicodedata
# import datetime
import os
import json
from os import listdir
import pandas as pd
pd.set_option('display.width', 10000)
# import dateutil.parser
# import metadata_funs
# import xlrd
# import datetime
from os.path import isfile, join
# import ntpath
# import uuid
import re

In [15]:
def read_dataset_excel():
#     ex_data_path = os.path.join(os.getcwd(),'metadata/manually_curated_metadata/ADRF Dataset Metadata_clean.xlsx')
    ex_data_path = '/Users/sophierand/RichContextMetadata/metadata/raw_data/ADRF Dataset Metadata_clean.xlsx'
    xls = pd.ExcelFile(ex_data_path)
    dataset_workbook = pd.read_excel(ex_data_path, sheet_name=None)
    return dataset_workbook

def read_class_metadata(dataset_workbook):
    uniform_sheets = ['course1-datasets','course2-datasets' ,'kcmo-datasets', 'in_data_2019-datasets'
    , 'mo_data_2019-datasets', 'usda-datasets']
    new_df_list = []
    for i in range(len(uniform_sheets)):
        a = dataset_workbook[uniform_sheets[i]]
        new_df_list.append(a)
    classes_df = pd.concat(new_df_list, sort = False).drop_duplicates()
    pattern = re.compile('([^\sa-zA-Z]|_)+')
    classes_df['alias'] = classes_df['title'].apply(lambda x: pattern.sub('',x).strip())
    classes_df['alias'] = classes_df['alias'].apply(lambda x: re.sub(' +', ' ',x))
    return classes_df

def read_bb_metadata(dataset_workbook):
    bb_df = dataset_workbook['bundesbank-rc']
    bb_df['alias'] = bb_df['dataset_join_id']
    return bb_df

def gen_dataset_metadata(bb_df,classes_df):
    common_fields = list(set(classes_df.columns.values) & set(bb_df.columns.values))
    bb_df_lim = bb_df[common_fields]
    classes_df_lim = classes_df[common_fields]
    final_df = pd.concat(list((bb_df_lim,classes_df_lim)), sort = False).drop_duplicates()
    return final_df


In [16]:
dataset_workbook = read_dataset_excel()
a = read_class_metadata(dataset_workbook)
b = read_bb_metadata(dataset_workbook)
f = gen_dataset_metadata(bb_df = b, classes_df = a)

In [14]:
adrf_metadata_path  =  '/Users/sophierand/RichContextMetadata/metadata/manually_curated_metadata/adrf_metadata.csv'
f.to_csv(adrf_metadata_path,header = True, index = False, sep = ',')